<a href="https://colab.research.google.com/github/Jey-Sankar-Sai-Pinjala/Jey-Sankar-Sai-Pinjala/blob/main/Spam_Detection_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd

In [42]:
import re

In [43]:
import nltk

In [44]:
from nltk.corpus import stopwords

In [45]:
from nltk.stem import PorterStemmer

In [46]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
stemmer = PorterStemmer()

In [48]:
stop_words = set(stopwords.words("english"))

In [49]:
df=pd.read_csv("/01. Project Spam Detection - DATASET.csv",encoding='latin-1') [["v1","v2"]]

In [50]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [51]:
df.tail()

,v1,v2
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [52]:
df.columns=['label','message']

In [53]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [54]:
df["label"]=df["label"].map({"ham":0,"spam":1})

In [55]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [56]:
def preprocess_text(text):
  text=re.sub(f"[^a-zA-Z0-9]"," ",text)
  text=text.lower()
  text=text.split()
  text=[stemmer.stem(word) for word in text if word not in stop_words]
  return " ".join(text)

In [57]:
df["cleaned_message"] = df["message"].apply(preprocess_text)

In [58]:
df.head()

,label,message,cleaned_message
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
from sklearn.linear_model import LogisticRegression

In [62]:
from sklearn.metrics import accuracy_score,classification_report

In [63]:
vectorizer=TfidfVectorizer(max_features=3000)
X=vectorizer.fit_transform(df["cleaned_message"])

In [64]:
y = df["label"]

In [65]:
x_train,x_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=42)

In [66]:
model=LogisticRegression()

In [67]:
model.fit(x_train,y_train)

LogisticRegression()

In [68]:
y_pred=model.predict(x_test)

In [69]:
print(f"accuracy score:{accuracy_score(y_test,y_pred)*100:.2f}%")

accuracy score:95.43%


In [70]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.96      0.69      0.80       150

    accuracy                           0.95      1115
   macro avg       0.96      0.84      0.89      1115
weighted avg       0.95      0.95      0.95      1115



In [71]:
def predict_email(email_text):
  processed_text=preprocess_text(email_text)
  processed_data=preprocess_text(email_text)
  vectorized_text=vectorizer.transform([processed_text])
  prediction=model.predict(vectorized_text)
  return "Spam" if prediction[0]==1 else "Ham - Not Spam"

In [72]:
email= "we are updated our login credential policy. Please confirm your account by logging into Google Docs."

In [73]:
print(f"Email:{email} /n Prediction:{predict_email(email)}")

Email:we are updated our login credential policy. Please confirm your account by logging into Google Docs. /n Prediction:Ham - Not Spam
